Alejandro Barrachina Argudo

Walid Bousnitra Bousnitra

# Práctica 1. Resolución de problemas con búsqueda 
## Inteligencia Artificial I    2022/2023

La práctica está organizada en 4 partes:
* Parte I: se muestra a través de ejemplos resueltos cómo se representan algunos problemas clásicos como el de las jarras, el problema de los misioneros o el problema del ocho puzzle. 
* Parte II: se muestra el uso de los algoritmos de búsqueda exhaustiva (ciega y heurística) vistos en clase. 
* Parte III: aprenderemos a medir las propiedades de los algoritmos.
* Parte IV: se os dan problemas algo más complejos para resolver al menos 1.

En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  
Cuando termines los ejercicios entrega **este archivo en el campus**.  
Debes incluir al comienzo una celda de markdown con el nombre completo de los miembros del grupo y número de grupo.

Los comentarios razonados de los ejercicios son la parte más importante de esta práctica. 

**Fecha de entrega de la práctica completa: 18 de octubre** 

## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en representar estados y acciones, la comprobación de objetivo y el coste de las acciones.
La siguiente clase Problem (de la librería AIMA) representa este esquema general de cualquier problema de espacio de estados. Un problema concreto será una subclase de Problem, y requerirá implementar
* estado_inicial, es_estado_final(_), acciones(_), aplica(_,_), coste_de_aplicar_accion y eventualmente __init__ 

El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. 


 __Lo más sencillo para empezar es copiar en tu directorio de trabajo los archivos _search.py y utils.py_ de AIMA__

In [1]:
# from search import Problem
import math

In [2]:
## Copiamos aquí la definición de la clase Problem de la librería AIMA para facilitar la explicación.

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError
        
    def print(self):
        print("Hola")

    def coste_de_aplicar_accion(self, estado, accion):
        """Respecto a la versiòn original de AIMA hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo __definir un problema como subclase
de problema__. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [4]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        #accs es una lista que inicializamos vacía, comprobaremos las precondiciones y añadiremos en esta lista las acciones aplicables al estado.
        accs=list() 
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
        if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs
        # se devuelve en accs todas las acciones aplicables

    def result(self,estado,accion):
    # aplica una acción a un estado (esta función se llamará desde el algoritmo de búsqueda)
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [5]:
# Estado inicial
p =Jarras()
p.initial

(0, 0)

In [6]:
# Acciones 
p.actions(p.initial)

['trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'llenar jarra de 3']

In [7]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [8]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [9]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [3]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
# Aunque no es necesario puedes hacer tu propia implementación del problema 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema. En este caso el estado inicial y objetivo se pasan como parámetros'''
        Problem.__init__(self, initial, goal)
        # En este caso representamos cada accion con un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido. La comprobación de las precondiciones se realiza en una función auxiliar is_valid
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3 
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:  #si la canoa está en el lado 0
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [7]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

In [8]:
misioneros.initial

(3, 3, 0)

In [9]:
# Acciones aplicables en el estado inicial
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

#Por ejemplo, para resolver el problema de los misioneros con 
#el método de busqueda en anchura la llamada sería:  
from search import *
breadth_first_tree_search(misioneros).solution()

In [4]:
from search import breadth_first_tree_search
breadth_first_tree_search(misioneros).solution()

NameError: name 'misioneros' is not defined

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [5]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):        
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
                
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES.
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        elif accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        elif accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0    

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [12]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [13]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [14]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [15]:
try:
    p8.result(p8.initial,"Mover hueco abajo")
except Exception as e:
    print(e)

list index out of range


In [16]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [17]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

### EJERCICIO 2.  Usando la representación anterior representa el problema del puzle de 16 piezas.

In [ ]:
class Quince_Puzzle(Problem): 
    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):        
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2,3):
            accs.append("Mover hueco arriba")
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if pos_hueco not in (12,13,14,15):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,4,8,12):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (3,7,11,15):
            accs.append("Mover hueco derecha")
                
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-4]
            l[pos_hueco-4] = 0
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES.
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+4]
            l[pos_hueco+4] = 0
        elif accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        elif accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
        # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
        #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        
        # Asumimos que el espacio N siempre es par

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        if(state.index(0) in [0,1,2,3,8,9,10,11] & inversion % 2 == 1):
            return True
        elif (state.index(0) in [4, 5,6,7,12,13,14,15] & inversion % 2 == 0):
            return True
        
        return False

Realiza algunas pruebas para comprobar que la clase está bien definidA P

In [19]:
p15 = Quince_Puzzle((3,6,5,12,7,2,8,14,9,15,1,0,4,10,11,13))
p15.initial

(3, 6, 5, 12, 7, 2, 8, 14, 9, 15, 1, 0, 4, 10, 11, 13)

In [20]:
p15.actions(p15.initial)

['Mover hueco arriba', 'Mover hueco abajo', 'Mover hueco izquierda']

In [21]:
p15.result(p15.initial,"Mover hueco arriba")

(3, 6, 5, 12, 7, 2, 8, 0, 9, 15, 1, 14, 4, 10, 11, 13)

In [22]:
p15.result(p15.initial,"Mover hueco izquierda")

(3, 6, 5, 12, 7, 2, 8, 14, 9, 15, 0, 1, 4, 10, 11, 13)

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

En primer lugar vamos a ver cómo funciona la búsqueda ciega (en anchura y en profundidad) para encontrar soluciones para los tres problemas anteriores: el problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [6]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

Para que la importación de search funcione el archivo search.py y utils.py tienen que estar en la misma carpeta que este notebook o poner la ruta correspondiente.

In [24]:
## comprueba la resolución del problema de las jarras con el método de búsqueda en anchura.  

In [25]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [26]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### EJERCICIO 3. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8 y de 16.  Comenta al final cómo se comportan los algoritmos en cuanto a eficiencia y resultados. 

In [27]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

#Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura 

# breadth_first_tree_search(misioneros).solution()

# Realiza pruebas con los diferentes algoritmos observando el comportamiento.

In [28]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [33]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [29]:
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
## La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
## Esta llamada no funciona, se queda en [*]
## Si es método de busqueda en anchura es completo.. ¿no debería terminar? ¿qué crees que está pasando?
## Podría ser porque se encuentra en le caso de que el enunciado(estado inicial) no tenga solución y por tanto el algoritmo
## esté moviendo el hueco indefinidamente.

KeyboardInterrupt: 

In [7]:
# Comprueba si el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) tiene solución.
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
if p8.check_solvability(p8.initial):
    sol= breadth_first_tree_search(p8).solution()
else: 
    sol="Este estado inicial no tiene solucion"

In [31]:
sol

'Este estado inicial no tiene solucion'

In [32]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [33]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [34]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [35]:
depth_first_graph_search(estado).solution()

['Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco derecha',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover h

In [41]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [43]:
p15 = Quince_Puzzle((1,2,3,4,5,6,8,9,10,7,11,12,13,14,15, 0))
p15.initial


(1, 2, 3, 4, 5, 6, 8, 9, 10, 7, 11, 12, 13, 14, 15, 0)

In [44]:
breadth_first_tree_search(p15).solution()

KeyboardInterrupt: 

### Comentarios del ejercicio 3. 
#### Se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones respecto a la resolución de los problemas con búsqueda ciega:
- Misioneros: 
  - La busqueda de solución por busqueda en anchura parece funcionar bastante rápido
- Puzle de 8: 
  - La busqueda de solución por busqueda en anchura en grafo y en anchura en arbol da resultados bastante rápido, si lo intentamos con un grafo en altura se queda buscando solución durante mucho rato
- Puzle de 16:  Mismos resultados que para Puzle de 8 pero tardando ligeramente más

**********************************************************************************
Puedes añadir todo el espacio que necesites.

### EJERCICIO 4:  Definición de heurísticas

In [45]:
%%time
breadth_first_tree_search(misioneros)

CPU times: user 146 ms, sys: 7.34 ms, total: 153 ms
Wall time: 159 ms


<Node (0, 0, 1)>

In [46]:
#### 4.1. Para el problema de los misioneros define una heurística y comenta sus propiedades
# Puedes utilizar alguna de las heurísticas vistas en clase. 

# Heurísticas para el problema de los misioneros.

def h1(node):
    """
        Devuelve el numero de personas a la izquierda por dos (dos viajes ya que alguien siempre vuelve) y resta uno si la barca está a la izquierda
    """
    goal = (0,0,1)
    misioneros_izq = node.state[0]
    canibales_izq = node.state[1]
    pos_barca = node.state[2]
    return 2 * (canibales_izq + misioneros_izq) - pos_barca

In [47]:
astar_search(misioneros, h1).solution()

[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [48]:
%%time
astar_search(misioneros, h1).solution()

CPU times: user 158 µs, sys: 0 ns, total: 158 µs
Wall time: 172 µs


[('1m1c', (1, 1)),
 ('1m', (1, 0)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [13]:
#### 4.2. Para el problema del puzle de 8 y de 16 se pide definir  las siguientes funciones heurísticas:
# linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
# manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
# max_heuristic(node: maximo de las dos anteriores
# sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 

def linear(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    mal_colocados = 0
    for i in range(len(goal)):
        if node.state.index(i) != goal.index(i):
            mal_colocados +=1
    return mal_colocados



def manhattan(node):
    state = node.state
    mhd = 0
    goal = (1,2,3,4,5,6,7,8,0)
    for i in range(len(state)):
        if(state.index(i) != goal.index(i)):
            mhd += int((abs(i -goal.index(state[i]))/3) + ((abs(i-goal.index(state[i]))))%3)
            # / 3 para saber la distancia vertical
            # % 3 para saber la distancia horizontal
    
    return mhd
                
def sqrt_manhattan(node):
    state = node.state
    
    return math.sqrt(manhattan(node))

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        

### EJERCICIO 5. 
#### Usar las implementaciones de AIMA de los algoritmos de búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema de los misioneros y algun ejemplo del puzle de 8 y puzle de 16. 

#### Comparar los costes temporales usando %timeit y comentar los resultados.

In [50]:
####  Vamos a probar el 8-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+

puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [51]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [52]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [ ]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [53]:
astar_search(puzle,linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [54]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? ¿y en los resultados?
Vamos a medirlo 

In [55]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [56]:
%%time
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

CPU times: user 2.64 ms, sys: 0 ns, total: 2.64 ms
Wall time: 2.66 ms


<Node (1, 2, 3, 4, 5, 6, 7, 8, 0)>

In [57]:
%%time
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

CPU times: user 4.07 ms, sys: 3.47 ms, total: 7.54 ms
Wall time: 7.07 ms


<Node (1, 2, 3, 4, 5, 6, 7, 8, 0)>

In [58]:
%%time
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

CPU times: user 42.9 ms, sys: 105 µs, total: 43 ms
Wall time: 43.1 ms


<Node (1, 2, 3, 4, 5, 6, 7, 8, 0)>

In [59]:
%%time
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

CPU times: user 5.28 ms, sys: 0 ns, total: 5.28 ms
Wall time: 5.21 ms


<Node (1, 2, 3, 4, 5, 6, 7, 8, 0)>

####  Realiza pruebas también con el puzle de 16 con los estados iniciales que quieras.  Escribe aquí tus conclusiones sobre qué heurística te parece mejor para este problema.  

Ten en cuenta que el número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. En el caso del siguiente estado inicial que sí tiene solución: (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


#### Justifica claramente la respuesta basándote en los resultados esperados  (según las clases teóricas) y los resultados obtenidos en las pruebas anteriores.

Basandonos en la experiencia del 8 puzzle, podemos concluir que la busqueda lineal solo nos va a dar mejores resultados para casos en los que la solución esté muy cercana, mientras la manhattan nos dará una busqueda más informada y por tanto hará una mayor poda.


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [8]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas. 

# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

In [9]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [38]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [63]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [64]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [65]:
astar_search(p8, manhattan).solution()

['Mover hueco derecha']

In [66]:
astar_search(p8p, manhattan).solution()

['Mover hueco derecha']

In [10]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """
    p8=Ocho_Puzzle(estado_inicial)
    p8p=Problema_con_Analizados(p8)
    if p8.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [68]:
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2


In [11]:
e1 = (7,2,4,3,6,1,8,0,5) 
e2 = (4,5,6,1,0,2,7,8,2)
e3 = (5,3,1,8,0,4,6,2,7)
e4 = (0,8,7,6,5,4,3,2,1)
e5 = (8,0,3,1,2,6,4,7,5)

In [70]:
%%time
resuelve_ocho_puzzle(e1,algoritmo=breadth_first_graph_search)

Este problema no tiene solucion. 
CPU times: user 61 µs, sys: 0 ns, total: 61 µs
Wall time: 63.2 µs


In [71]:
%%time
resuelve_ocho_puzzle(e2,algoritmo=breadth_first_graph_search)

Este problema no tiene solucion. 
CPU times: user 677 µs, sys: 181 µs, total: 858 µs
Wall time: 653 µs


In [72]:
%%time
resuelve_ocho_puzzle(e3,algoritmo=breadth_first_tree_search)


: 

: 

In [42]:
%%time
resuelve_ocho_puzzle(e4, algoritmo=breadth_first_graph_search)


Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 28. Nodos analizados: 178224
CPU times: user 13min 30s, sys: 36.1 ms, total: 13min 30s
Wall time: 13min 34s


In [43]:
%%time
resuelve_ocho_puzzle(e5,algoritmo=breadth_first_tree_search)


: 

: 

In [13]:
%time resuelve_ocho_puzzle(e1, depth_first_tree_search)
%time resuelve_ocho_puzzle(e2, depth_first_tree_search)
%time resuelve_ocho_puzzle(e3, depth_first_tree_search)
%time resuelve_ocho_puzzle(e4, depth_first_tree_search)
%time resuelve_ocho_puzzle(e5, depth_first_tree_search)


Este problema no tiene solucion. 
CPU times: user 52 µs, sys: 0 ns, total: 52 µs
Wall time: 53.6 µs
Este problema no tiene solucion. 
CPU times: user 17 µs, sys: 0 ns, total: 17 µs
Wall time: 19.1 µs


: 

: 

In [ ]:
%time resuelve_ocho_puzzle(e1,uniform_cost_search)
%time resuelve_ocho_puzzle(e2,uniform_cost_search)
%time resuelve_ocho_puzzle(e3,uniform_cost_search)
%time resuelve_ocho_puzzle(e4,uniform_cost_search)
%time resuelve_ocho_puzzle(e5,uniform_cost_search)

Este problema no tiene solucion. 
Este problema no tiene solucion. 


KeyboardInterrupt: 

In [ ]:
%time resuelve_ocho_puzzle(e1,best_first_graph_search, linear)
%time resuelve_ocho_puzzle(e2,best_first_graph_search, linear)
%time resuelve_ocho_puzzle(e3,best_first_graph_search, linear)
%time resuelve_ocho_puzzle(e4,best_first_graph_search, linear)
%time resuelve_ocho_puzzle(e5,best_first_graph_search, linear)


Este problema no tiene solucion. 
Este problema no tiene solucion. 
Solución: ['Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco abajo', 'Move

In [ ]:
%time resuelve_ocho_puzzle(e1,best_first_graph_search, manhattan)
%time resuelve_ocho_puzzle(e2,best_first_graph_search, manhattan)
%time resuelve_ocho_puzzle(e3,best_first_graph_search, manhattan)
%time resuelve_ocho_puzzle(e4,best_first_graph_search, manhattan)
%time resuelve_ocho_puzzle(e5,best_first_graph_search, manhattan)

Este problema no tiene solucion. 
Este problema no tiene solucion. 
Solución: ['Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco i

In [15]:
%time  resuelve_ocho_puzzle(e1,astar_search, linear)
%time resuelve_ocho_puzzle(e2,astar_search, linear)
%time resuelve_ocho_puzzle(e3,astar_search, linear)
%time resuelve_ocho_puzzle(e4,astar_search, linear)
%time resuelve_ocho_puzzle(e5,astar_search, linear)

Este problema no tiene solucion. 
CPU times: user 710 µs, sys: 5 µs, total: 715 µs
Wall time: 524 µs
Este problema no tiene solucion. 
CPU times: user 0 ns, sys: 55 µs, total: 55 µs
Wall time: 57.9 µs
Solución: ['Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 26. Nodos analizados: 34703
CPU times: user 5min 21s, sys: 88.7 ms, total: 5min 21s
Wall time: 5min 22s


In [14]:
%time resuelve_ocho_puzzle(e1,astar_search, manhattan)
%time resuelve_ocho_puzzle(e2,astar_search, manhattan)
%time resuelve_ocho_puzzle(e3,astar_search, manhattan)
%time resuelve_ocho_puzzle(e4,astar_search, manhattan)
%time resuelve_ocho_puzzle(e5,astar_search, manhattan)

Este problema no tiene solucion. 
CPU times: user 1.36 ms, sys: 29 µs, total: 1.39 ms
Wall time: 3.37 ms
Este problema no tiene solucion. 
CPU times: user 29 µs, sys: 3 µs, total: 32 µs
Wall time: 35 µs
Solución: ['Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 26. Nodos analizados: 9760
CPU times: user 32.3 s, sys: 53.6 ms, total: 32.4 s
Wall time: 32.6 s
Solución: ['Mover hueco

### Ejercicio 6:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

          E1              E2              E3              E4               E5
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+  
     | 7 | 2 | 4 |   | 4 | 5 | 6 |   | 5 | 3 | 1 |   | H | 8 | 7 |    | 8 | H | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 3 | 6 | 1 |   | 1 | 0 | 3 |   | 8 | 0 | 4 |   | 6 | 5 | 4 |    | 1 | 2 | 6 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 8 | H | 5 |   | 7 | 8 | 2 |   | 6 | 2 | 7 |   | 3 | 2 | 1 |    | 4 | 7 | 5 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo)
 
-----------------------------------------------------------------------------------------
                                       E1           E2           E3          E4        E5
    Anchura                            L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
   
    Profundidad                        L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
                                       
    Coste Uniforme                     L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T=
                                       NA=          NA=          NA=         NA=       NA=
                                       
    Primero el mejor (linear)          L=           L=           L=          L=        L=
                                       T=           T=           T=          T=        T= 
                                       NA=          NA=          NA=         NA=       NA= 
                                                                            
    Primero el mejor (manhattan)       L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                             
    A* (linear)                        L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                             
    A* (manhattan)                     L=           L=           L= 26       L= 28
                                       T=           T=           T= 1'2"     T= 6,92"
                                       NA=          NA=          NA= 9760    NA= 3444


 -----------------------------------------------------------------------------------------
PUEDES AMPLIAR LA TABLA CON PRUEBAS ADICIONALES. 

Escribe aquí tus conclusiones 

 **justificar los resultados con las distintas propiedades teóricas estudiadas en clase**.  



### Ejercicio 7:  resolver usando distintas búsquedas y/o heurísticas, el problema del 16 puzzle para algunos  estados iniciales:

Escribe aquí tus conclusiones sobre los resultados y la eficiencia de las distintas opciones. 



In [ ]:
def linear2(node):
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
    mal_colocados = 0
    for i in range(len(goal)):
        if node.state.index(i) != goal.index(i):
            mal_colocados +=1
    return mal_colocados



def manhattan2(node):
    state = node.state
    mhd = 0
    goal = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
    for i in range(len(state)):
        if(state.index(i) != goal.index(i)):
            mhd += int((abs(i -goal.index(state[i]))/4) + ((abs(i-goal.index(state[i]))))%4)
            # / 4 para saber la distancia vertical
            # % 4 para saber la distancia horizontal
    
    return mhd
                
def sqrt_manhattan2(node):
    state = node.state
    
    return math.sqrt(manhattan2(node))

def max_heuristic2(node):
    score1 = manhattan2(node)
    score2 = linear2(node)
    return max(score1, score2)

def resuelve_quince_puzzle(estado_inicial, algoritmo, h=None):
    p15=Quince_Puzzle(estado_inicial)
    p15p=Problema_con_Analizados(p15)
    if p15.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p15p,h).solution()
        else: 
            sol= algoritmo(p15p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p15p.analizados))
    else: 
        print("Este problema no tiene solucion. ")

In [ ]:
e6 = (1,2,3,4,5,6,7,8,9,10,11,12,13,14,0,15)

In [ ]:
%%time
resuelve_quince_puzzle(e6,astar_search, linear2)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear2
Longitud de la solución: 1. Nodos analizados: 2
CPU times: total: 0 ns
Wall time: 1.6 ms


###  PARTE IV:  Se os proporcionará un enunciado de algún problema para representar y resolver usando el paradigma del espacio de estados. 


## Ejercicio: Muelle de carga


In [ ]:
class Muelles(Problem):
    def __init__(self, initial, goal=None):
        super().__init__(initial, goal)
    
    def actions(self, state):
        

In [ ]:
m = Muelles((3,[[1,0], [0,0], [0,0]], [0,1], [0,1,0], [0]), (3, [], []))